In [1]:
import os

import matplotlib.pyplot as plt
import nivapy3 as nivapy
import numpy as np
import pandas as pd

plt.style.use("ggplot")

# Assessment of water quality trends

Øyvind and Marianne would like trend tests for the main Numedalslågen stations. See e-mail from Marianne received 14.08.2025 for details.

 * Stations and data of interest are provided by Marianne in the file `vannmiljo_data_mainstations.xlsx`.

 * Parameters `N-NO3_F_µg/l N` and `N-NO3_T_µg/l N` can be combined into a single variable.
   
 * Parameters `N-TOT_F_µg/l N` and `N-TOT_T_µg/l N` can be combined into a single variable.

To reduce autocorrelation and remove season effects, I will calculate **annual median concentrations** and perform both Mann-Kendall and Sen's slope tests on the annual data.

In [2]:
# Data from Marianne
xl_path = r"../data/vannmiljo_data_mainstations.xlsx"

# Parameters of interest
pars = [
    "N-NO3_µg/l N",
    "N-TOT_µg/l N",
    "P-PO4_F_µg/l P",
    "P-PO4_T_µg/l P",
    "P-TOT_F_µg/l P",
    "P-TOT_T_µg/l P",
    "TOC_F_mg/l C",
]

res_dir = r"../results"

In [3]:
# Read raw data
df = pd.read_excel(xl_path, sheet_name="main-st_N-P-TOC")
df["year"] = df["sample_date"].dt.year

# Merge columns
df["N-NO3_µg/l N"] = df[["N-NO3_F_µg/l N", "N-NO3_T_µg/l N"]].mean(axis="columns")
df["N-TOT_µg/l N"] = df[["N-TOT_F_µg/l N", "N-TOT_T_µg/l N"]].mean(axis="columns")

# Get data of interest
id_cols = ["station_id", "station_code", "station_name", "year"]
df = df[id_cols + pars]

# Annual medians
df = df.groupby(id_cols).median().reset_index()

df.head()

,station_id,station_code,station_name,year,N-NO3_µg/l N,N-TOT_µg/l N,P-PO4_F_µg/l P,P-PO4_T_µg/l P,P-TOT_F_µg/l P,P-TOT_T_µg/l P,TOC_F_mg/l C
0,28202,015-28202,Pikerfossen. Numedalslågen,1977,68.5,230.0,NaN,NaN,6.0,NaN,NaN
1,28202,015-28202,Pikerfossen. Numedalslågen,1978,70.0,200.0,NaN,NaN,5.0,NaN,NaN
2,28202,015-28202,Pikerfossen. Numedalslågen,1979,75.0,220.0,NaN,NaN,6.5,NaN,NaN
3,28202,015-28202,Pikerfossen. Numedalslågen,1980,65.0,190.0,NaN,NaN,5.0,NaN,NaN
4,28202,015-28202,Pikerfossen. Numedalslågen,1981,51.5,195.0,NaN,NaN,6.5,NaN,NaN


In [4]:
stats_dict = {
    "station_id": [],
    "station_code": [],
    "station_name": [],
    "parameter": [],
    "start_year": [],
    "end_year": [],
    "n_values": [],
    "max": [],
    "min": [],
    "mean": [],
    "median": [],
    "mk_trend": [],
    "mk_pvalue": [],
    "sens_trend": [],
    "sens_slope": [],
}
for stn_id, stn_df in df.groupby("station_id"):
    stn_code = stn_df["station_code"].iloc[0]
    stn_name = stn_df["station_name"].iloc[0]
    print(f"\n{stn_code}")
    for par in pars:
        print(f"  {par}")
        par_df = (
            stn_df[["year", par]].dropna(subset=par).sort_values("year", ascending=True)
        )

        if len(par_df) == 0:
            print("    WARNING: No data.")
            stats_dict["station_id"].append(stn_id)
            stats_dict["station_code"].append(stn_code)
            stats_dict["station_name"].append(stn_name)
            stats_dict["parameter"].append(par)
            stats_dict["start_year"].append(np.nan)
            stats_dict["end_year"].append(np.nan)
            stats_dict["n_values"].append(np.nan)
            stats_dict["min"].append(np.nan)
            stats_dict["max"].append(np.nan)
            stats_dict["mean"].append(np.nan)
            stats_dict["median"].append(np.nan)
            stats_dict["mk_trend"].append(np.nan)
            stats_dict["mk_pvalue"].append(np.nan)
            stats_dict["sens_trend"].append(np.nan)
            stats_dict["sens_slope"].append(np.nan)

        elif len(par_df) < 10:
            print("    WARNING: Fewer than 10 annual values. Skipping trend test.")
            stats_dict["station_id"].append(stn_id)
            stats_dict["station_code"].append(stn_code)
            stats_dict["station_name"].append(stn_name)
            stats_dict["parameter"].append(par)
            stats_dict["start_year"].append(par_df["year"].min())
            stats_dict["end_year"].append(par_df["year"].max())
            stats_dict["n_values"].append(len(par_df))
            stats_dict["min"].append(par_df[par].min())
            stats_dict["max"].append(par_df[par].max())
            stats_dict["mean"].append(par_df[par].mean())
            stats_dict["median"].append(par_df[par].median())
            stats_dict["mk_trend"].append(np.nan)
            stats_dict["mk_pvalue"].append(np.nan)
            stats_dict["sens_trend"].append(np.nan)
            stats_dict["sens_slope"].append(np.nan)

        else:
            # MK test
            mk_df = nivapy.stats.mk_test(par_df, par)

            # Sen's slope
            res_df, sen_df = nivapy.stats.sens_slope(
                par_df, value_col=par, index_col="year"
            )

            # Trend plots
            nivapy.plotting.plot_sens_slope(
                res_df, sen_df, ylabel=par, title=f"{stn_name} ({stn_code})"
            )
            par_minus_unit = par[:-7]
            png_path = os.path.join(res_dir, "trend_plots", f"{stn_code}_{par_minus_unit}.png")
            plt.savefig(png_path, dpi=200, bbox_inches="tight")
            plt.close()

            stats_dict["station_id"].append(stn_id)
            stats_dict["station_code"].append(stn_code)
            stats_dict["station_name"].append(stn_name)
            stats_dict["parameter"].append(par)
            stats_dict["start_year"].append(par_df["year"].min())
            stats_dict["end_year"].append(par_df["year"].max())
            stats_dict["n_values"].append(len(par_df))
            stats_dict["min"].append(par_df[par].min())
            stats_dict["max"].append(par_df[par].max())
            stats_dict["mean"].append(par_df[par].mean())
            stats_dict["median"].append(par_df[par].median())
            stats_dict["mk_trend"].append(mk_df.loc["trend", "value"])
            stats_dict["mk_pvalue"].append(mk_df.loc["p", "value"])
            stats_dict["sens_trend"].append(res_df.loc["trend", "value"])
            stats_dict["sens_slope"].append(res_df.loc["sslp", "value"])

            print("    Done.")

stats_df = pd.DataFrame(stats_dict)

# Save
res_path = os.path.join(res_dir, 'main_stations_trend_summary.xlsx')
stats_df.to_excel(res_path, index=False)

stats_df.head()


015-28202
  N-NO3_µg/l N
  N-TOT_µg/l N
    Done.
  P-PO4_F_µg/l P
  P-PO4_T_µg/l P
  P-TOT_F_µg/l P
    Done.
  P-TOT_T_µg/l P
  TOC_F_mg/l C
    Done.

015-28488
  N-NO3_µg/l N
  N-TOT_µg/l N
    Done.
  P-PO4_F_µg/l P
  P-PO4_T_µg/l P
  P-TOT_F_µg/l P
    Done.
  P-TOT_T_µg/l P
  TOC_F_mg/l C
    Done.

015-28489
  N-NO3_µg/l N
  N-TOT_µg/l N
    Done.
  P-PO4_F_µg/l P
  P-PO4_T_µg/l P
  P-TOT_F_µg/l P
    Done.
  P-TOT_T_µg/l P
  TOC_F_mg/l C
    Done.

015-28490
  N-NO3_µg/l N
  N-TOT_µg/l N
    Done.
  P-PO4_F_µg/l P
  P-PO4_T_µg/l P
  P-TOT_F_µg/l P
    Done.
  P-TOT_T_µg/l P
  TOC_F_mg/l C
    Done.

015-28510
  N-NO3_µg/l N
  N-TOT_µg/l N
    Done.
  P-PO4_F_µg/l P
  P-PO4_T_µg/l P
  P-TOT_F_µg/l P
    Done.
  P-TOT_T_µg/l P
  TOC_F_mg/l C
    Done.

015-28526
  N-NO3_µg/l N
  N-TOT_µg/l N
    Done.
  P-PO4_F_µg/l P
  P-PO4_T_µg/l P
  P-TOT_F_µg/l P
    Done.
  P-TOT_T_µg/l P
  TOC_F_mg/l C

015-28527
  N-NO3_µg/l N
  N-TOT_µg/l N
    Done.
  P-PO4_F_µg/l P
  P-PO4_T_µg/l P
 

,station_id,station_code,station_name,parameter,start_year,end_year,n_values,max,min,mean,median,mk_trend,mk_pvalue,sens_trend,sens_slope
0,28202,015-28202,Pikerfossen. Numedalslågen,N-NO3_µg/l N,1977.0,1983.0,7.0,75.0,51.5,63.928571,65.0,NaN,NaN,NaN,NaN
1,28202,015-28202,Pikerfossen. Numedalslågen,N-TOT_µg/l N,1977.0,2024.0,29.0,231.5,145.0,193.224138,194.0,no trend,0.910341,no trend,-0.03596
2,28202,015-28202,Pikerfossen. Numedalslågen,P-PO4_F_µg/l P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,28202,015-28202,Pikerfossen. Numedalslågen,P-PO4_T_µg/l P,2006.0,2006.0,1.0,2.0,2.0,2.000000,2.0,NaN,NaN,NaN,NaN
4,28202,015-28202,Pikerfossen. Numedalslågen,P-TOT_F_µg/l P,1977.0,2024.0,29.0,7.0,2.0,4.863793,4.5,decreasing,0.000452,decreasing,-0.05000
